### Load the libraries

In [1]:
# Load the libraries 
import os, sys
import pandas as pd 
import numpy as np 
import scanpy as sc
import anndata as ad 
from pyensembl import EnsemblRelease
import sqlite3

/usr/local/lib/python3.10/dist-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/usr

<i><b> Print the container version </b></i>

In [2]:
# Container used for this analysis can be found here : cokorac/cs-core-image-amd64:dev (date of use : 24/01/25)

<i><b> Set the home directory </b></i>

In [3]:
# Define the home_path 
os.environ['HOME_Nikola_scdgomics'] = "/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scdgomics"
home_path = os.getenv("HOME_Nikola_scdgomics")
home_path

'/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scdgomics'

### Load the data

In [4]:
# Load the Polioudakis and Trevino raw counts
adata_human_neocortex = sc.read_h5ad("/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scmorpho_final/data_versions_tissue/Adata_raw.h5ad")
# Load the Linnarsson raw counts
adata_mouse_dg = sc.read_h5ad("/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scdgomics/data_versions/Adata_raw.h5ad")

/usr/local/lib/python3.10/dist-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
# Convert mouse gene names from lower to uppercase
adata_mouse_dg.var_names.str.upper()

Index(['0610007P14RIK', '0610009B22RIK', '0610009L18RIK', '0610009O20RIK',
       '0610010F05RIK', '0610010K14RIK', '0610011F06RIK', '0610012D04RIK',
       '0610012G03RIK', '0610025J13RIK',
       ...
       'MT-CO2', 'MT-CO3', 'MT-CYTB', 'MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4',
       'MT-ND4L', 'MT-ND5', 'MT-ND6'],
      dtype='object', name='cellid', length=27933)

### Data wrangling

> *** NOTE: Find the downloaded ensembl releases for human and mouse genomes here : '/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scdgomics/ensembl_release_104'

In [6]:
# Download the ENSEMBL RELEASE 104 

## human

#data = EnsemblRelease(104)
#data.download()
#data.index()
#all_genes = data.genes()
#protein_coding_hs = [gene.gene_name for gene in all_genes if gene.biotype == 'protein_coding']
#len(protein_coding_hs)

## mouse

#data = EnsemblRelease(104, species='mus_musculus')
#data.download()
#data.index()
#all_genes = data.genes()
#protein_coding_mm = [gene.gene_name for gene in all_genes if gene.biotype == 'protein_coding']
#len(protein_coding_hs)

In [7]:
# Import GTF files and keep only protein-coding genes
## human 
human_gtf_path = '/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scdgomics/ensembl_release_104/GRCh38/ensembl104/Homo_sapiens.GRCh38.104.gtf.db'
connection_hs = sqlite3.connect(human_gtf_path)
df_hs = pd.read_sql_query("SELECT * FROM gene;", connection_hs)
df_protein_coding_hs = df_hs[df_hs['gene_biotype'] == 'protein_coding']
## mouse
mouse_gtf_path = '/group/kalebic/Nikola/1_single_cell_final/Nikola_final/final/scdgomics/ensembl_release_104/GRCm39/ensembl104/Mus_musculus.GRCm39.104.gtf.db'
connection_mm = sqlite3.connect(mouse_gtf_path)
df_mm = pd.read_sql_query("SELECT * FROM gene;", connection_mm)
df_protein_coding_mm = df_mm[df_mm['gene_biotype'] == 'protein_coding']

In [8]:
# Print the number of protein-coding genes
print(len(df_protein_coding_hs))
print(len(df_protein_coding_mm))

19966
21885


In [9]:
# Keep only protein coding genes
## human
adata_human_neocortex = adata_human_neocortex[:, adata_human_neocortex.var_names.isin(df_protein_coding_hs['gene_name'].tolist())]
## mouse 
adata_mouse_dg = adata_mouse_dg[:, adata_mouse_dg.var_names.isin(df_protein_coding_mm['gene_name'].tolist())]

In [10]:
# Add the dataset_of_origin column to Linnarsson dataset
adata_human_neocortex.obs['dataset_of_origin'] = 'Linnarsson'

/tmp/ipykernel_3156017/2263678302.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_human_neocortex.obs['dataset_of_origin'] = 'Linnarsson'


In [11]:
# Concatenate the raw datasets 
adata_combined = ad.concat([adata_human_neocortex, adata_mouse_dg], join = 'outer')
adata_combined.var_names_make_unique()

### Data normalization

In [12]:
# Create the layer for raw counts
adata_combined.layers['counts'] = adata_combined.X.toarray().copy()

In [13]:
# Normalize counts
sc.pp.normalize_total(adata_combined, target_sum=1e4, exclude_highly_expressed=True)
adata_combined.layers['normalized_counts'] = adata_combined.X.copy()

In [14]:
# Log-transform
sc.pp.log1p(adata_combined)
adata_combined.layers['log_normalized_counts'] = adata_combined.X.copy()

In [15]:
# Identify HVGs
sc.pp.highly_variable_genes(adata_combined, subset=False, batch_key = 'dataset_of_origin')

... storing 'dataset_of_origin' as categorical
... storing 'Sample name (24185 single cells)' as categorical
... storing 'SRR run accession' as categorical
... storing 'raw file (original file name)' as categorical
... storing 'UMI_CellularBarcode' as categorical


In [16]:
# Scale
sc.pp.scale(adata_combined, max_value=10)
adata_combined.layers['log_normalized_scaled_counts'] = adata_combined.X.copy()

### Save the data

In [17]:
adata_combined.write_h5ad(os.path.join(home_path, 'data_versions/Adata_normalized.h5ad'))